# AutoML Classifier

This is a component that performs predictions using an AutoML Classifier implementation from [autosklearn](https://github.com/automl/auto-sklearn). 
<br>
auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.

This notebook shows:
- how to use SDK to load a model.
- how to use a model to provide real-time predictions.

In [ ]:
%%writefile Predictor.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model, load_dataset
from platiagro.featuretypes import CATEGORICAL, NUMERICAL, infer_featuretypes
from sklearn.exceptions import NotFittedError 
from sklearn.utils.validation import check_is_fitted

logger = logging.getLogger(__name__)


class Predictor(object):
    """
    Model template. You can load your model parameters in __init__ from a location accessible at runtime.
    """

    def __init__(self, dataset=None, target=None, experiment_id=None):
        logger.info("Initializing")

        # Load Estimator and Encoders
        model = load_model(experiment_id=experiment_id)
        self.estimator = model["estimator"]
        self.feature_encoder = model["feature_encoder"]
        self.label_encoder = model["label_encoder"]

        # Load Dataset and Feature Types
        df = load_dataset(name=dataset)
        columns = df.columns.tolist()
        target_idx = columns.index(target)
        featuretypes = infer_featuretypes(df)

        # Compute replacement for NaN values
        # mode for categorical features
        # mean for numerical features.
        categorical = [columns[idx] for idx, ft in enumerate(featuretypes) if ft == CATEGORICAL and idx != target_idx]
        numerical = [columns[idx] for idx, ft in enumerate(featuretypes) if ft == NUMERICAL and idx != target_idx]
        self.categorical_nan_replacement = df.loc[:, categorical].mode(axis=0)
        self.numerical_nan_replacement = df.loc[:, numerical].mean(axis=0)

        logger.info("Init complete!")

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Returns a prediction.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.

        Returns:
            Array-like with predictions.
        """
        # Encode categorical features
        try:
            check_is_fitted(self.feature_encoder, "categories_")
            X = self.feature_encoder.transform(X)
        except NotFittedError:
            pass

        # Put data in a pandas.DataFrame
        df = pd.DataFrame(X, columns=feature_names)

        # Replace NaNs
        df.fillna(self.categorical_nan_replacement, inplace=True)
        df.fillna(self.numerical_nan_replacement, inplace=True)

        # Perform Prediction
        y_pred = self.estimator.predict(df.to_numpy())

        # Apply Label Decoder
        return self.label_encoder.inverse_transform(y_pred)

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%sh
MODEL_NAME=Predictor
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0
LOG_LEVEL=DEBUG
PARAMETERS='[{"type":"STRING","name":"dataset","value":"iris"},{"type":"STRING","name":"target","value":"Species"},{"type":"STRING","name":"experiment_id","value":"48f2668a-e31a-4b5a-a91a-28fd649f7adc"}]'

seldon-core-microservice $MODEL_NAME $API_TYPE \
    --service-type $SERVICE_TYPE \
    --persistence $PERSISTENCE \
    --parameters $PARAMETERS \
    --log-level $LOG_LEVEL > log.txt 2>&1 &

until $(curl --output /dev/null --silent --head --fail http://localhost:5000/health/ping); do
    printf '.'
    sleep 1
done

## Make predictions

In [ ]:
!curl localhost:5000/predict -d 'json={"data":{"names":["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], "ndarray":[[5.1, 3.5, 1.4, 0.2]]}}'

## View logs

In [ ]:
!cat log.txt

## Cleans up the test

In [ ]:
!ps -ef | grep [s]eldon-core-mic | awk '{print $2}' | xargs -r kill